# Number Representation


## Integers

Integer numbers are represented by N bit words. Python3 allows you to store integers with practically **unlimited precision**, the only limitation comes from the (contiguous) space available in memory.
In Python2, N depends on the PC architercture, N=64 in modern computers.

In [ ]:
import sys

# size of an int "0"
a = 0
print(sys.getsizeof(a))

# size of an int "100"
a = 100
print(sys.getsizeof(a))

# size of an int "2**64"
a = 2**64
print(sys.getsizeof(a))

In [ ]:
# Check the largest integer
print(sys.maxsize)

# Check also that corresponds to a 64-bit integer
print("Is your system a 64 bit one?", 2**63 - 1 == sys.maxsize)

# Python3 doesn't have a limit for integers
maxint = sys.maxsize+1
print(maxint)

### Binary representation

The common assumption is that numbers (in Python as in all the other languages) are expressed as decimal numbers. Built-in functions allows explicitly to convert from one base to another.  

In the binary representation, typically 1 bit ($j$) is used to specify the sign of the number, and the conversion between binary and decimal representation is:

$$d = (-1)^j\sum_{i=0}^{N-1} \alpha_i ~ 2^i$$

where $\alpha_i$ are either 0 or 1. 
$b=\alpha_{N-1}\alpha_{N-2}...\alpha_0$ is the binary representation of the number.

Example: an 8-bit integer in binary representation with one bit for the sign:

|  j | 6 | 5 | 4 | 3 | 2 | 1 | 0  |
|---|---|---|---|---|---|---|---|
|  0 | 0 | 0 | 1 | 0 | 1 | 1 | 1  |

corresponds to:

$$d = (-1)^j\sum_{i=0}^{N-1} \alpha_i ~ 2^i = (-1)^{0} [ (1) \cdot 2^0 + (1) \cdot 2^1 + (1) \cdot 2^2 + (0) \cdot 2^3 + (1) \cdot 2^4 + (0) \cdot 2^5 + (0) \cdot 2^6] = 1 + 2 + 4 + 16 = 23$$


### Hexadecimal representation

When dealing with long binary numbers, it's convenient to group bits by groups of **four** and convert them to the hexadecimanl (hex) representation (base 16). In hex base, numbers are represented by a digit from 0-F.

| hex | bin |
| --- | --- |
| 0 | 0000 |
| 1 | 0001 |
| 2 | 0010 |
| 3 | 0011 |
| 4 | 0100 |
| 5 | 0101 |
| 6 | 0110 |
| 7 | 0111 |
| 8 | 1000 |
| 9 | 1001 |
| A | 1010 |
| B | 1011 |
| C | 1100 |
| D | 1101 |
| E | 1110 |
| F | 1111 |

**Example**: colors are commonly saved using 24-bits, for example:

`10001010 10111001 11100011`

The three bytes (8-bits blocks) specify the red, green, and blue components (RGB). Each one is composed of two blocks of 4 bits, i.e. an hex digit. The same 24-bit word is thus equivalent to:

`#8AB9E3`

which is indeed referred to as "Hex color".

Switching between dec, bin and hex representations in Python is straightforward using the casting functions:

In [ ]:
# an integer in decimal representation
a = 23

# its binary representation
a_bin = bin(a)
print('Binary representation of', a, ':', a_bin)

# its hexadecimal representation
a_hex = hex(a)
print('Hexadecimal representation of', a, ':', a_hex)

# converting back to integer
print('Decimal representation of', a_bin, ':', int(a_bin, 2))
print('Decimal representation of', a_hex, ':', int(a_hex, 16))

### Bitwise operators

#### Logical operators

In [ ]:
a = 60           # 60 = 0011 1100 
b = 13           # 13 = 0000 1101 

c = a & b        # 12 = 0000 1100
print("Bitwise AND ", c)

c = a | b        # 61 = 0011 1101 
print("Bitwise OR  ", c)

c = a ^ b        # 49 = 0011 0001
print("Bitwise XOR ", c)

#### Unary operator

The `~` operator returns the complement of a number, which is the number you get by switching each 1 for a 0 and each 0 for a 1. This is the same as `-x - 1`

In [ ]:
c = ~a           # -61 = 1100 0011
print("Bitwise NOT ", c)

#### Shifts

In [ ]:
c = a << 2       # 240 = 1111 0000
print("Left shift (towards most significant) of two positions ", c)

c = a >> 2       # 15 = 0000 1111
print("Right shift (towards least significant) of two positions ", c)

Link to the Python [documentation](https://realpython.com/python-bitwise-operators/).

#### Masking and shifting

Logical operators and shifts can be combined to "read" a bitstream, by filtering long binary words.

**Example**: a 8-bit word is used to store 3 integer values. Starting from the leftmost bit:
 - the first number uses 1 bit
 - the second number uses 4 bits
 - the third number the remaining 3 bits.
We want to extract these 3 values from the initial word.

|  bit number  | 7 | 6 | 5 | 4 | 3 | 2 | 1 | 0  |
|---|---|---|---|---|---|---|---|--- |
| first integer | X | | | | | | | |
| second integer | | X | X | X | X | | | |
| third integer | | | | | | X | X | X |
|  binary word (206) | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0  |

 - first number (1 bit): first, we create a mask (an appropriate binary word) that filters only the information of the desired bit, and sets to 0 everything else
   
|  binary word (206) | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0  |
|---|---|---|---|---|---|---|---|--- |
|  mask (128)| 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0  |
|  masked word (128) | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0  |

then, we shift the resulting word by the appropriate amount of bits, in this case:

|  masked word (128) | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0  |
|---|---|---|---|---|---|---|---|--- |
|  masked and shifted (1) | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1  |

The Python code to perform these operations is:

In [ ]:
word = 206
first_mask = 128
first_shift = 7
first_number = (word & first_mask) >> first_shift
print(first_number)

 - second number (4 bits):
 
 |  bit number  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8  |
|---|---|---|---|---|---|---|---|--- |
|  binary word (206) | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0  |
|  mask (120)| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0  |
|  masked word (64) | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0  |
|  masked and shifted (9) | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1  |

In [ ]:
second_mask = 120
second_shift = 3
second_number = (word & second_mask) >> second_shift
print(second_number)

## Floating point numbers

Non-integer number **cannot be represented with infinite precision** on a computer. Single precision (also known as *float*) and double precision numbers use 32 and 64 bits respectively. 
Note that all floating point numbers in Python are double precision (64 bits).

A standard has been developed by IEEE (IEEE-754) to represent floating point numbers in hardware such that the relative precision (see later) is the same across the whole validity range.

The 32 or 64 bits are divided among 3 quantities uniquely characterizing the number:

$x_{float} = (-1)^s \times 1.f \times 2^{e-bias}$

where:
 - *s* is the sign
 - *f* the fractional part of the mantissa
 - *e* the exponent.
 
In addition, in order to get numbers smaller than 1, a constant *bias* term is added to the exponent. Such *bias* is typically equal to **half of the max value of *e***.

The mantissa is defined as:

${\rm mantissa}=1.f=1+\frac{m_{n-1}}{2^{1}}+\frac{m_{n-2}}{2^{2}}+..+\frac{m_{0}}{2^{n}}$

where $n$ is the number of bits dedicated to *f* (see below) and $m_i$ are the binary coefficients. 

Numbers exceeding the maximum allowed value are *overflows* and the calculations involving them provide incorrect answers. Numbers smaller in absolute value than the minimum allowed value are *underflows* and simply set to zero, also in this case incorrect results are yielded.  

### Single precision

For single precision floating point numbers, $0\le e \le 255$ and $bias=127$. Bits are arranged as follows:

|   | *s* | *e* | *f* |
|---|---|---|---|
| Number of bits | 1 | 8 | 23 |
| Bit position | 31 | 30-23 | 22-0 |

An example is given below:

In [ ]:
from IPython.display import Image
Image(url='http://www.dspguide.com/graphics/F_4_2.gif')

Special values are also possibiles. N.B.: those are not numbers that can be used in the mathematical sense!

|  special value |  conditions | value |
|---|---|---|
|  $+\infty$ | s=0, e=255, f=0 | +INF  |
|  $-\infty$ | s=1, e=255, f=0 | -INF  |
|  not a number | e=255, f>0  | NaN  |

The largest value is obtained for $f\sim 2$ and $e=254$, i.e. $2\times2^{127}\sim 3.4\times10^{38}$.

The value closest to zero is obtained instead for $f=2^{-23}$ and $e=0$, i.e. $2^{-149}\sim 1.4\times10^{-45}$.


### Double precision

For double precision floating point numbers, $0\le e \le 2047$ and $bias=1023$. Bits are arranged as follows:

|   | *s* | *e* | *f* |
|---|---|---|---|
| Number of bits | 1 | 11 | 52 |
| Bit position | 63 | 62-52 | 51-0 |

Special values are also possibiles. N.B.: those are not numbers that can be used in the mathematical sense!

|  special value |  conditions | value |
|---|---|---|
|  $+\infty$ | s=0, e=2047, f=0 | +INF  |
|  $-\infty$ | s=1, e=2047, f=0 | -INF  |
|  not a number | e=2047, f>0  | NaN  |

The validity range for double numbers is from $2.2 \times 10^{-308}$ up to $1.8 \times 10^{308}$

Serious scientific calculations almost always requires at least double precision floating point numbers.

#### Floating point numbers on your system

Information about the floating point reresentation on your system can be obtained from `sys.float_info`. Definitions of the stored values are given on the Python doc [page](https://docs.python.org/3/library/sys.html#sys.float_info).

In [ ]:
import sys
print(sys.float_info)

## Accuracy and the perils of calculations with floats


Floats can only have a limited number of meaningful decimal places, on the basis of how many bits are allocated for the fractional part of the mantissa: 6-7 decimal places for singles, 15-16 for doubles.

This means that calculations involving numbers with more than those decimal places involved do not yield the correct result, simply because the binary representation of those numbers does not allow to store them with sufficient accuracy.

In [ ]:
# Addding an increasingly small number to 7
for e in [14, 15, 16]:
    print(7 + 1.0 * 10**-e)

There are ways to print floats (e.g. filling data into an output file) controlling the number of decimals:

In [ ]:
print(format(math.pi, '.13f'))  # give 13 significant digits

print('%.14f' % (0.1 * 0.1 * 100)) # give <15 significant digits
print('%.17f' % (0.1 * 0.1 * 100)) # give >15 significant digits

This is not a bug, but the direct consequence to the fact that the mantissa is represented by a limited amount of bits, therefore calculations can only make sense if an appropriate number of decimal digits are concerned:

In [ ]:
# 23 bits are used for f in single precision floating point
print("Single precision:", 2**-23)

# 53 bits are used for f in double precision floating point
print("Double precision:", 2**-53)

### Operations with floats

It should never been forgotten that computers store numbers in binary format. In the same way it is not possible to express the fraction 1/3 with a finite decimal places, analogously fractions that are well represented in the decimal base cannot be easily represented in binary, e.g. 1/10 is the infinitely repeating number:

$0.0001100110011001100110011001100110011001100110011...$

corresponding to $3602879701896397/2^{55}$ which is close to but not exactly equal to the true value of 1/10 (even though it is even printed to be like that!).
Similarly 0.1 is not 1/10, and making calculations assuming that exactly typically yield to wrong results:

In [ ]:
# sometimes, trivial (for humans) operations can yield unexpected results:
print(0.1 + 0.1 == 0.2)

# does it work for 0.3, too?
print(0.1 + 0.1 + 0.1 == 0.3)

A lesson of paramount importance is that you must **never** compare floating point numbers with the "==" operator as *what is printed is not what is stored*!

The function ```float.hex()``` yield the exact value stored for a floating point number:

In [ ]:
import math
x = math.pi
print("dec =", x)
print("hex =", x.hex())

# from the previous example: the two numbers are not the same, bit-wise
print((0.1 + 0.1 + 0.1).hex(), (0.3).hex())

For finite floating-point numbers, this representation will always include a leading `0x` and a trailing `p` and exponent. Since Python’s floats are stored internally as binary numbers, converting a float to or from a decimal string usually involves a small rounding error.

In contrast, hexadecimal strings allow exact representation and specification of floating-point numbers. This can be useful when debugging, and in numerical work.

A typical case is subtraction of numbers very close by in value (e.g. when dealing with spectral frequencies). The same happens with functions evaluated near critical points (see later):

In [ ]:
print(1 + 6.022e23 - 6.022e23)

In these cases, associative law does not necessarily hold:

In [ ]:
print(6.022e23 - 6.022e23 + 1)
print(1 + 6.022e23 - 6.022e23)

Distributive law does not hold:

In [ ]:
import math
a = math.exp(1)
b = math.pi
c = math.sin(1)
a*(b + c) == a*b + a*c

Also identities after casting large numbers may not yield the expected result

In [ ]:
x = 287475839859383374
print(x == int(float(x)))

## From numbers to functions: conditioning and stability

#### Function conditioning

A mathematical function $f(x)$ is well-conditioned if $f(x+\epsilon)\simeq f(x)$ for all small perturbations $\epsilon$.

In other words, the function $f(x)$ is **well-conditioned** if the solution varies gradually as the input varies. For a well-conditioned function, small pertubations in the input result in small effects in the output. However, a poorly-conditioned problem only needs some small perturbation to have large effects. For example, inverting a nearly singluar matrix (a matrix whose determinant is close to zero) is a poorly conditioned problem.

#### Algorithm stability

Suppose we have a computer algorithm $g(x)$ that implements the mathematical function $f(x)$. $g(x)$ is **numerically stable** if $g(x+\epsilon) \simeq f(x)$ and it is called **unstable** if large changes in the output are produced.

Analyzing an algorithm for stability is more complicated than determining the condition of an expression, even if the algorithm simply evaluates the expression. This is because an algorithm consists of many basic calculations and each one must be analyzed and, due to roundoff error, we must consider the possibility of small errors being introduced in every computed value.

Numerically unstable algorithms tend to amplify approximation errors due to computer arithmetic over time. If we used an infinite precision numerical system, stable and unstable alorithms would have the same accuracy. However, as we see below (e.g. variance calculation), when using floating point numbers, algebrically equivalent algorithms can give different results.

In general, we need both a well-conditioned problem and an algorihtm with sufficient numerical stabilty to obtain reliably accurate answers. In this case, we can be sure that $g(x) \simeq f(x)$.

In most of the cases, the solution to stability issues is solved by properly redefining the function as in the example above and below.

#### In summary:

- Well-/ill-conditioned refers to the problem; Stable/Unstable refers to an algorithm or numerical process.
- If the problem is well-conditioned, then there is a stable way to solve it.
- If the problem is ill-conditioned, then there is no reliable way to solve it in a stable way.
- Mixing roundoff-error with an unstable process is a recipe for disaster.
- With exact arithmetic (no roundoff-error), stability is not a concern.

1. Example of a poorly conditioned function: the tangent of an angle

In [ ]:
import math
# Define two numbers x and x + epsilon very close to pi/2
x1 = 1.57078
x2 = 1.57079
# Calculate the tangent of the x1 and x2 angles
t1 = math.tan(x1)
t2 = math.tan(x2)

print ('tan(x1) =', t1)
print ('tan(x2) =', t2)
print ('% change in x =', 100.0*(x2-x1)/x1, '%')
print ('% change in tan(x) =', (100.0*(t2-t1)/t1), '%')

2. Example of a numerically unstable algorithm: the limit      $\lim_{x \to 0} \frac{1-\cos(x)}{x^2}$

In [ ]:
# Catastrophic cancellation occurs when subtracitng
# two numbers that are very close to one another

# We'll see numpy and matplotlib in the next lectures: forget about the technical details, for now
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return (1 - np.cos(x))/(x*x)

#x = np.linspace(-4e-1, 4e-1, 1000) # uncomment to zoom out
x = np.linspace(-4e-8, 4e-8, 1000)
plt.plot(x, f(x))
plt.axvline(1.1e-8, color='red')

In [ ]:
# We know from L'Hopital's rule that the answer is 0.5 at 0
# and should be very close to 0.5 throughout this tiny interval
# but errors arisee due to catastrophic cancellation

print('%.30f' % np.cos(1.1e-8))
print('%.30f' % (1 - np.cos(1.1e-8))) # failure point: the exact answer is 6.05e-17
print('%2f' % ((1 - np.cos(1.1e-8))/(1.1e-8*1.1e-8)))

Solution: rewrite the function using $\sin$ instead of $\cos$: $1-\cos(x)$ = $2 \sin^2 (\frac{x}{2})$

In [ ]:
# Numerically stable version of funtion using simple trignometry

def f1(x):
    return 2*np.sin(x/2)**2/(x*x)

#x = np.linspace(-4e-1, 4e-1, 1000) # uncomment to zoom out
x = np.linspace(-4e-8, 4e-8, 1000)
plt.plot(x, f1(x))
plt.axvline(1.1e-8, color='red')

3. Another common example of a numerically unstable algorithm. The stable and unstable version of the variance:

$s^2 = \frac{1}{n-1} \sum (x-\bar{x})^2$

In [ ]:
# check the result of the calculation of the variance of an array
# of randomly distributed data between [0, 1] around 1e12
x = 1e12 + np.random.uniform(0, 1, int(1e3))

# direct method
# squaring occuring after subtraction, element-wise
def direct_var(x):
    n = len(x)
    xbar = np.mean(x)
    return 1.0/(n-1)*np.sum((x - xbar)**2)

# sum of squares method (vectorized version)
# pay attention to the subtraction of two large numbers
def sum_of_squares_var(x):
    n = len(x)
    return (1.0/(n*(n-1))*(n*np.sum(x**2) - (np.sum(x))**2))

# Welford's method
# an optimized method
def welford_var(x):
    s = 0
    m = x[0]
    for i in range(1, len(x)):
        m += (x[i] - m) / i
        s += (x[i] - m)**2
    return s/(len(x) - 1)


# correct answer from a purpose-built function in numpy
print("Numpy:", np.var(x))
print("Direct:", direct_var(x))
print("Sum of squares:", sum_of_squares_var(x))
print("Welford's:", welford_var(x))

4. The example of the Likelihood: $\mathcal{L} = \prod_{i=0}^{N} Poisson(x, \mu)$

In [ ]:
# loss of precision can be a problem when calculating Likelihoods
probs = np.random.random(1000) # Generating 1000 random numbers between 0 and 1, as if they were probabilities
#print(probs)
print("L =", np.prod(probs))

# when multiplying lots of small numbers, work in log space
print("log L =", np.sum(np.log(probs)))